In [32]:
import pandas as pd
import numpy as np
import redcap

API_Token = ''
project = redcap.Project('https://redcap.prc.utexas.edu/redcap/api/', API_Token)

data_csv_path = '/Users/md35727/Downloads/data (18)/data.csv'
data_folder_path = '/Users/md35727/Downloads/data (18)/'
participant_identifier_var = "participant_info:1"

participant_info = pd.read_csv(data_csv_path, index_col= False)

In [30]:
for i in range(len(participant_info.index)):
    #is participant data already in REDCap
    duplicate_participant = False
    participant_id = participant_info.loc[i, participant_identifier_var]
    textfile = participant_info.loc[i, "igt_experiment:1"]
    subset = project.export_records(fields=['participant_id', 'iowa_gambling_task_igt_complete'])
    for s in subset:
        if (str(s['participant_id']) == str(participant_id) and s['iowa_gambling_task_igt_complete'] == '1'):
            duplicate_participant = True
            print('Participant ' + str(participant_id) + ' is already in REDCap.')
    if not duplicate_participant:
        with open(data_folder_path + textfile) as myfile:
            lines = list(myfile) 
            igt_choices = []
            igt_holdings = lines[99].split(" ")[2]
            igt_disadvantageous = 0
            igt_advantageous = 0
            igt_frequent_loss = 0
            igt_infrequent_loss = 0
            for i in range(len(lines)):   
                current_line = lines[i].split(" ")
                if(current_line[1] == '1'):
                    igt_disadvantageous+=1
                    igt_frequent_loss+=1
                elif(current_line[1] == '2'):
                    igt_disadvantageous+=1
                    igt_infrequent_loss+=1
                elif(current_line[1] == '3'):
                    igt_advantageous+=1
                    igt_frequent_loss+=1
                elif(current_line[1] == '4'):
                    igt_advantageous+=1
                    igt_infrequent_loss+=1
                igt_choices.append(current_line[1])
                igt_ltc_index = igt_advantageous - igt_disadvantageous
                igt_ifl_index = igt_infrequent_loss - igt_frequent_loss
            data_dict = {'participant_id': int(participant_id), 
                 'igt_101': igt_holdings, 'igt_102': igt_advantageous,
                 'igt_103': igt_disadvantageous, 'igt_104': igt_frequent_loss, 
                 'igt_105': igt_infrequent_loss, 'igt_106': igt_ltc_index, 
                 'igt_107': igt_ifl_index, 'iowa_gambling_task_igt_complete': 1}
            for i in range(len(igt_choices)):
                if(i < 9):
                    data_dict['igt_0' + str(i+1)] = igt_choices[i]
                elif(i >= 9):
                    data_dict['igt_' + str(i+1)] = igt_choices[i]
            import_data = project.import_records([data_dict])
            print('Participant ' + str(participant_id) + ' data has been added to REDCap.')


Participant 2 is already in REDCap.
Participant 1 is already in REDCap.
